In [3]:
from __future__ import print_function

#torch
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#torchvision
import torchvision.models as models

#image
from PIL import Image
import cv2

#jupyter
from ipywidgets import FloatProgress
from IPython.display import display


#os
import os
import os.path as path
import glob

#math
import math
import numpy as np
import random

In [4]:
def channel_shuffle(x, groups):
    batchsize, num_channels, height, width = x.data.size()

    channels_per_group = num_channels // groups
    x = x.view(batchsize, groups, channels_per_group, height, width) #add a new dimension
    
    x = torch.transpose(x, 1, 2).contiguous()
    return x.view(batchsize, -1, height, width)

In [5]:
class GroupNorm(nn.Module):
    def __init__(self, num_features, num_groups=32, eps=1e-5):
        super(GroupNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(1,num_features,1,1))
        self.bias = nn.Parameter(torch.zeros(1,num_features,1,1))
        self.num_groups = num_groups
        self.eps = eps

    def forward(self, x):
        N,C,H,W = x.size()
        G = self.num_groups
        assert C % G == 0

        x = x.view(N,G,-1)
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, keepdim=True)

        x = (x-mean) / (var+self.eps).sqrt()
        x = x.view(N,C,H,W)
        return x * self.weight + self.bias

In [6]:
class IdentityBloc(nn.Module):
    def __init__(self,input_channels, output_channels, nb_groups, nb_frames=1):
        super(IdentityBloc, self).__init__()
        self.conv3 = nn.Conv2d(input_channels*nb_frames, input_channels*nb_frames, kernel_size=3, stride=1, padding=1, groups=nb_groups*nb_frames)
        self.bn3 = nn.BatchNorm2d(input_channels*nb_frames)
        self.conv1 = nn.Conv2d(input_channels*nb_frames, output_channels*nb_frames, kernel_size=1, stride=1, padding=0, groups=nb_frames)
        self.bn1 = nn.BatchNorm2d(output_channels*nb_frames)
        self.relu = nn.ReLU(True)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        right = self.relu(self.bn3(self.conv3(x)))
        right = self.relu(self.bn1(self.conv1(right)))
        return x.add(right)

In [7]:
class DownBloc(nn.Module):
    def __init__(self,input_channels, output_channels, nb_groups, nb_frames=1, shuffle=False):
        super(DownBloc, self).__init__()
        self.nb_groups = nb_groups
        self.shuffle   = shuffle
        self.conv3     = nn.Conv2d(input_channels*nb_frames, input_channels*nb_frames, 
                                   kernel_size=3, stride=2, padding=1, groups=nb_groups*nb_frames)
        self.bn3       = nn.BatchNorm2d(input_channels*nb_frames)
        self.conv1     = nn.Conv2d(input_channels*nb_frames, output_channels*nb_frames, kernel_size=1, stride=1, padding=0, groups=nb_frames)
        self.bn1       = nn.BatchNorm2d(output_channels*nb_frames)
        self.relu      = nn.ReLU()
        self.pool      = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        left = self.pool(x)
        right = self.relu(self.bn3(self.conv3(x)))
        if self.shuffle:
            right = channel_shuffle(right, self.nb_groups)
        right = self.relu(self.bn1(self.conv1(right)))
        return torch.cat( (left, right), 1)

In [8]:
class FusionBloc(nn.Module):
    def __init__(self,input_channels, output_channels, nb_frames=1):
        super(FusionBloc, self).__init__()
        self.conv1     = nn.Sequential(
                            nn.Conv2d(input_channels*nb_frames, output_channels, kernel_size=1, stride=1, padding=0),
                            nn.BatchNorm2d(output_channels),
                            nn.ReLU()
                            )
        
    def forward(self, x):
        return self.conv1(x)

In [9]:
class DenseMobile(nn.Module):
    def __init__(self, in_channel=3, nb_frames=1, num_classes=6, first_group=True):
        super(DenseMobile, self).__init__()
        if first_group:
            self.blocStart = DownBloc(in_channel, 32-in_channel, in_channel, nb_frames=nb_frames) #224x224x3 -> 112x112x32
        else:
            self.blocStart = nn.Sequential(
                            nn.Conv2d(in_channel, 32, kernel_size=3, stride=2, padding=1),
                            nn.MaxPool2d(kernel_size=3, padding=1, stride=1)
                        )
        self.iddown = nn.Sequential(
                        IdentityBloc(32, 32, 32, nb_frames=nb_frames), # -> 112x112x32
                        DownBloc(32,32,32, nb_frames=nb_frames), # -> 56x56x64
                        IdentityBloc(64,64,64, nb_frames=nb_frames), # -> 56x56x128
                        DownBloc(64, 192, 64, nb_frames=nb_frames), # -> 28x28x256
                        FusionBloc(256,256, nb_frames),
                        IdentityBloc(256,256, 256), # -> 28x28x256
                        DownBloc(256, 256, 256), # -> 14x14x512
                        IdentityBloc(512, 512, 512)
                    )
        

        self.end = nn.Sequential(
                        DownBloc(512,512,512), # -> 7x7x1024
                        nn.AvgPool2d(kernel_size=7, padding=0, stride=1), #3x3
        )
        self.classif = nn.Linear(1024, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        x = self.blocStart(x)
        x = self.iddown(x)
        #x1 = self.dense(x)
        #x.add(x1)
        x = self.end(x)
        return self.classif(x.view(x.size(0),-1))

In [14]:
class DenseMobileF1(nn.Module):
    def __init__(self, num_classes=6, first_group=True):
        super(DenseMobileF1, self).__init__()
        
        self.blocStart = DownBloc(9, 32-9, 9) #224x224x3 -> 112x112x32

        self.iddown = nn.Sequential(
                        IdentityBloc(32, 32, 32), # -> 112x112x32
                        DownBloc(32,32,32), # -> 56x56x64
                        IdentityBloc(64,64,64), # -> 56x56x128
                        DownBloc(64, 192, 64), # -> 28x28x256
                        IdentityBloc(256,256, 256), # -> 28x28x256
                        DownBloc(256, 256, 256), # -> 14x14x512
                        IdentityBloc(512, 512, 512)
                    )
        self.end = nn.Sequential(
                        DownBloc(512,512,512), # -> 7x7x1024
                        nn.AvgPool2d(kernel_size=7, padding=0, stride=1), #3x3
        )
        self.classif = nn.Linear(1024, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        x = self.blocStart(x)
        x = self.iddown(x)
        #x1 = self.dense(x)
        #x.add(x1)
        x = self.end(x)
        return self.classif(x.view(x.size(0),-1))

In [39]:
class DenseMobileF2(nn.Module):
    def __init__(self, num_classes=6, first_group=True):
        super(DenseMobileF2, self).__init__()
        
        self.blocStart = DownBloc(3, 32-3, 3, nb_frames=3) #224x224x3 -> 112x112x32

        self.iddown = nn.Sequential(
                        IdentityBloc(32, 32, 32, nb_frames=3), # -> 112x112x32
                        DownBloc(32,32,32, nb_frames=3), # -> 56x56x64
                        FusionBloc(64*3,64),
                        IdentityBloc(64,64,64), # -> 56x56x128
                        DownBloc(64, 192, 64), # -> 28x28x256
                        #FusionBloc(256,256),
                        IdentityBloc(256,256, 256), # -> 28x28x256
                        DownBloc(256, 256, 256), # -> 14x14x512
                        IdentityBloc(512, 512, 512)
                    )
        self.end = nn.Sequential(
                        DownBloc(512,512,512), # -> 7x7x1024
                        nn.AvgPool2d(kernel_size=7, padding=0, stride=1), #3x3
        )
        self.classif = nn.Linear(1024, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        x = self.blocStart(x)
        x = self.iddown(x)
        #x1 = self.dense(x)
        #x.add(x1)
        x = self.end(x)
        return self.classif(x.view(x.size(0),-1))

In [43]:
class DenseMobileF4(nn.Module):
    def __init__(self, num_classes=6, first_group=True):
        super(DenseMobileF4, self).__init__()
        
        self.blocStart = DownBloc(3, 32-3, 3, nb_frames=3) #224x224x3 -> 112x112x32

        self.iddown = nn.Sequential(
                        IdentityBloc(32, 32, 32, nb_frames=3), # -> 112x112x32
                        DownBloc(32,32,32, nb_frames=3), # -> 56x56x64
                        IdentityBloc(64,64,64, nb_frames=3), # -> 56x56x128
                        DownBloc(64, 192, 64, nb_frames=3), # -> 28x28x256
                        #FusionBloc(256,256),
                        IdentityBloc(256,256, 256, nb_frames=3), # -> 28x28x256
                        DownBloc(256, 256, 256, nb_frames=3), # -> 14x14x512
                        FusionBloc(512*3, 512),
                        IdentityBloc(512, 512, 512)
                    )
        self.end = nn.Sequential(
                        DownBloc(512,512,512), # -> 7x7x1024
                        nn.AvgPool2d(kernel_size=7, padding=0, stride=1), #3x3
        )
        self.classif = nn.Linear(1024, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        x = self.blocStart(x)
        x = self.iddown(x)
        #x1 = self.dense(x)
        #x.add(x1)
        x = self.end(x)
        return self.classif(x.view(x.size(0),-1))

In [46]:
class DenseMobileF5(nn.Module):
    def __init__(self, num_classes=6, first_group=True):
        super(DenseMobileF5, self).__init__()
        
        self.blocStart = DownBloc(3, 32-3, 3, nb_frames=3) #224x224x3 -> 112x112x32

        self.iddown = nn.Sequential(
                        IdentityBloc(32, 32, 32, nb_frames=3), # -> 112x112x32
                        DownBloc(32,32,32, nb_frames=3), # -> 56x56x64
                        IdentityBloc(64,64,64, nb_frames=3), # -> 56x56x128
                        DownBloc(64, 192, 64, nb_frames=3), # -> 28x28x256
                        #FusionBloc(256,256),
                        IdentityBloc(256,256, 256, nb_frames=3), # -> 28x28x256
                        DownBloc(256, 256, 256, nb_frames=3), # -> 14x14x512
                        IdentityBloc(512, 512, 512, nb_frames=3)
                    )
        self.end = nn.Sequential(
                        DownBloc(512,512,512, nb_frames=3), # -> 7x7x1024
                        nn.AvgPool2d(kernel_size=7, padding=0, stride=1), #3x3
        )
        self.classif = nn.Linear(1024*3, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x):
        x = self.blocStart(x)
        x = self.iddown(x)
        #x1 = self.dense(x)
        #x.add(x1)
        x = self.end(x)
        return self.classif(x.view(x.size(0),-1))

In [47]:
if __name__ == '__main__':
    #t = Variable(torch.Tensor(8,9,224,224))
    #bd = DownBloc(3, 32, 3, nb_frames=3)
    #print(bd(t).size())
    t = Variable(torch.Tensor(8,9,224,224))
    m = DenseMobileF4()
    print(m(t).size())

torch.Size([8, 6])
